# Dataset Mocks

> mocks for dataset

In [ ]:
#| default_exp mocks.adata

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass, field, KW_ONLY
from beartype.typing import Optional
from beartype import beartype
from litds.types import DataFrame

import pandas as pd, numpy as np
import anndata as ad

In [ ]:
#| export
from iza.static import TIME, SERIES, PHATE, BARCODE, CONDITION

In [ ]:
#| export
from litds.abc.mock.base import BaseMock

## Mock

### AnnData

In [ ]:
#| export
@dataclass
class MockSingleCell(BaseMock):
    label_col: Optional[str] = BARCODE
    condition_col: Optional[str] = CONDITION
    norm_str: Optional[str] = 'magic'
    
    n_genes: Optional[int] = 100
    n_points: Optional[int] = 1000
    emb_name: Optional[str] = PHATE
    emb_dims: Optional[int] = 3
    n_conditions: Optional[int] = 4
    hvg_ratio: Optional[int] = 5

    _: KW_ONLY
    
    seed: Optional[int] = 3
    set_index: Optional[bool] = False

    def setup(self) -> DataFrame:
        return self.adata

    def __post_init__(self):
        np.random.seed(self.seed)
        adata = self.setup()
        self.adata = adata
        
    
    @property
    def emb_cols(self):
        return self.make_col_names(self.emb_dims, prefix=f'{self.emb_name}_')
        
    @property
    def gene_symbols(self):
        try:
            return self._gene_symbols
        
        except AttributeError:
            _str = 'gene_symbol'
            name = f'{_str}s'
            func = lambda i: f'{_str} {i}'
            series = pd.Series(np.arange(self.n_genes), name=name).map(func)            
            self._gene_symbols = series
        return self._gene_symbols
    
    @property
    def is_hvg_gene(self):
        try:
            return self._is_hvg
        except AttributeError:
            is_hvg = pd.Series(np.arange(self.n_genes), name='is_hvg').map(lambda i: i % self.hvg_ratio)
            is_hvg.index = self.gene_symbols
            self._is_hvg = is_hvg
        return self._is_hvg
    
    @property
    def labels(self) -> pd.Series:
        try:
            return self._labels
        except AttributeError:
            _str = self.label_col
            name = f'{_str}s'
            func = lambda i: f'{_str} {i}'
            series = pd.Series(np.arange(self.n_points), name=name).map(func)
            self._labels = series
        return self._labels
    

    @property
    def conditions(self) -> pd.Series:
        try:
            return self._conditions
        except AttributeError:
            _str = self.condition_col
            name = f'{_str}s'
            func = lambda i: f'{_str} {i % self.n_conditions}'
            series = pd.Series(np.arange(self.n_points), name=name).map(func)
            series.index = self.labels
            self._conditions = series
        return self._conditions
    
    @property
    def df_emb(self) -> pd.DataFrame:
        try:
            return self._df_emb
        except AttributeError:
            df_emb = pd.DataFrame(
                np.random.randn(self.n_points, self.emb_dims), 
                index=self.labels, columns=self.emb_cols
            )
            df_emb = df_emb.join(self.conditions)
            self._df_emb = df_emb
        return self._df_emb
    

    @property
    def df_cnt(self) -> pd.DataFrame:
        try:
            return self._df_cnt
        except AttributeError:
            df_cnt = pd.DataFrame(
                np.random.randn(self.n_points, self.n_genes), 
                index=self.labels, columns=self.gene_symbols
            )
            self._df_cnt = df_cnt
        return self._df_cnt
    

    @property
    def df_nrm(self) -> pd.DataFrame:
        try:
            return self._df_nrm
        except AttributeError:
            df_nrm = pd.DataFrame(
                np.random.randn(self.n_points, self.n_genes), 
                index=self.labels, columns=self.gene_symbols
            )
            self._df_nrm = df_nrm
        return self._df_nrm
    
    @property
    def x_emb(self):
        return f'X_{self.emb_name.lower()}'
    
    @property
    def x_norm(self):
        return f'X_{self.norm_str.lower()}'
    
    @property
    def adata(self):
        try:
            return self._adata
        
        except AttributeError:
            _str = self.label_col
            label_name = f'{_str}s' # barcodes

            _str = self.condition_col
            conds_name = f'{_str}s' # conditions

            obs = self.df_emb.reset_index()
            obs = obs[[label_name, conds_name]].set_index(label_name, drop=False)

            obsm = dict()
            obsm[self.x_emb] = self.df_emb.drop(columns=conds_name)

            layers = dict()
            layers[self.x_norm] = self.df_nrm

            adata = ad.AnnData(
                X = self.df_cnt,
                obs = obs,
                var = pd.DataFrame(self.is_hvg_gene, index=self.gene_symbols),
                obsm = obsm,
                layers = layers,
                dtype='float32'
            )
            
            self._adata = adata

        return self._adata
    
    @adata.setter
    def adata(self, adata):
        self._adata = adata
        

In [ ]:
#| eval: False
mock = MockSingleCell()
mock.adata

AnnData object with n_obs × n_vars = 1000 × 100
    obs: 'barcodes', 'conditions'
    var: 'is_hvg'
    obsm: 'X_phate'
    layers: 'X_magic'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()